In [1]:
import pandas as pd

In [2]:
days = 186
year = '2023'
qualifying_pa = 503

In [3]:
# Get players that met the PA threshold
season_stats = pd.read_csv(f'./2023 Stats/stats_{year}_{days-1}.csv')

In [4]:
season_stats_qual = season_stats[season_stats['PA'] >= qualifying_pa]
qual_ids = season_stats_qual['mlbID']
qual_ids

0      682928
1      547989
3      660670
4      642715
12     645277
        ...  
624    664774
627    572233
648    677951
653    592885
655    807799
Name: mlbID, Length: 132, dtype: int64

In [5]:
player_dfs = {}

for day in range(0, days):
    # Load csv of league wide stats through the day
    date_stats = pd.read_csv(f'./2023 Stats/stats_{year}_{day}.csv')
    
    # Filter for players that met the PA threshold (from Fangraphs)
    date_stats_qual = date_stats[date_stats['mlbID'].isin(qual_ids)]

    # Add new column for day
    date_stats_qual['Day'] = day

    # Loop over each player in dataframe
    for player_id in date_stats_qual['mlbID'].unique():
        # Filter for player
        player_stats = date_stats_qual[date_stats_qual['mlbID'] == player_id]

        # If player already has dataframe append the new stats
        if player_id in player_dfs:
            player_dfs[player_id] = pd.concat([player_dfs[player_id], player_stats])
        else:
            player_dfs[player_id] = player_stats


C:\Users\16086\AppData\Local\Temp\ipykernel_12684\2942168442.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_stats_qual['Day'] = day
C:\Users\16086\AppData\Local\Temp\ipykernel_12684\2942168442.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_stats_qual['Day'] = day
C:\Users\16086\AppData\Local\Temp\ipykernel_12684\2942168442.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [6]:
player_dfs[682928]

,Name,Age,#days,Lev,Date,Tm,,Opp,G,PA,...,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlbID,Day
0,CJ Abrams,22,309,Maj-NL,"Mar 30, 2023",Washington,NaN,Atlanta,1,4,...,0,0,0,0,0.000,0.000,0.000,0.000,682928,0
0,CJ Abrams,22,309,Maj-NL,"Mar 30, 2023",Washington,NaN,Atlanta,1,4,...,0,0,0,0,0.000,0.000,0.000,0.000,682928,1
0,CJ Abrams,22,309,Maj-NL,NaN,Washington,NaN,NaN,1,4,...,0,0,0,0,0.000,0.000,0.000,0.000,682928,2
0,CJ Abrams,22,307,Maj-NL,NaN,Washington,NaN,NaN,2,7,...,0,0,0,0,0.000,0.000,0.000,0.000,682928,3
0,CJ Abrams,22,307,Maj-NL,NaN,Washington,NaN,NaN,2,7,...,0,0,0,0,0.000,0.000,0.000,0.000,682928,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,CJ Abrams,22,129,Maj-NL,NaN,Washington,NaN,NaN,147,594,...,3,7,43,4,0.245,0.301,0.416,0.717,682928,181
0,CJ Abrams,22,128,Maj-NL,NaN,Washington,NaN,NaN,148,598,...,3,7,44,4,0.245,0.301,0.415,0.716,682928,182
0,CJ Abrams,22,128,Maj-NL,NaN,Washington,NaN,NaN,148,598,...,3,7,44,4,0.245,0.301,0.415,0.716,682928,183
0,CJ Abrams,22,126,Maj-NL,NaN,Washington,NaN,NaN,149,603,...,3,7,44,4,0.243,0.298,0.411,0.710,682928,184
